## A. Import packages

In [1]:
import numpy as np
import re
from pathlib import Path
from glob import glob
from scipy import ndimage
from CellTracker.stardistwrapper import load_2d_slices_at_time

def extract_activities(tracked_labels_path: str, raw_path: str | dict, offset_value: float = 100.0, do_normalize: bool = False):
    tracked_labels_path = Path(tracked_labels_path)
    filenames = sorted(glob(str(tracked_labels_path / "labels_t=1-224_r20_v2_t*_z*.tif")))
    assert len(filenames) > 0, f"No label files were found in {tracked_labels_path}"

    # === t番号だけを抽出しユニークにソート ===
    t_numbers = sorted(set(
        int(re.search(r't(\d{6})', f).group(1))
        for f in filenames if re.search(r't(\d{6})', f)
    ))

    activity_matrix = None
    for idx, t in enumerate(t_numbers):
        print(f"t={t}...", end="\r")
        try:
            raw = load_2d_slices_at_time(raw_path, t=t, do_normalize=do_normalize)
        except FileNotFoundError:
            print(f"Warning: Raw images at t={t} cannot be loaded! Stop calculation!")
            break

        try:
            labels_img = load_2d_slices_at_time(str(tracked_labels_path / "labels_t=1-224_r20_v2_t%03i_z*.tif"), t=t, do_normalize=False)
        except FileNotFoundError:
            print(f"Warning: Label images at t={t} cannot be loaded!")
            if idx == 0:
                print("Warning: stop calculation!")
                return None
            else:
                print(f"Warning: skip volume {t}!")
                activity_matrix[idx, :] = np.nan
                continue

        if activity_matrix is None:
            cell_num = int(np.max(labels_img))
            activity_matrix = np.zeros((len(t_numbers), cell_num))

        found_bbox = ndimage.find_objects(labels_img, max_label=cell_num)
        for label in range(1, cell_num + 1):
            bbox = found_bbox[label - 1]
            if bbox is not None:
                intensity_values = raw[bbox][labels_img[bbox] == label] - offset_value
                intensity_values = intensity_values[intensity_values > 0]
                activity_matrix[idx, label - 1] = np.mean(intensity_values) if len(intensity_values) > 0 else np.nan
            else:
                activity_matrix[idx, label - 1] = np.nan

    return activity_matrix


ModuleNotFoundError: No module named 'CellTracker'

In [ ]:
import os
import numpy as np

# === 共通設定 ===
path_results = r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=517~565all"
tracked_labels_path = r"P:\Tardigrade\Tardigrada_mukumoto\orig\raw_t=517-565\labels_517-565"
os.makedirs(path_results, exist_ok=True)

# === 1. mCherry チャネル（channel 1） ===
raw_path_ch1 = {
    "h5_file": r"P:\Tardigrade\Tardigrada_mukumoto\orig\raw_t=517-565\raw_t=517-565.h5",
    "dset": "default",
    "channel": 0
}
activity_ch1 = extract_activities(
    tracked_labels_path=tracked_labels_path,
    raw_path=raw_path_ch1,
    offset_value=100.0,
    do_normalize=False
)
csv_ch1 = os.path.join(path_results, "tardigrade_t=517-565_signals_mCherry_offset100.csv")
timings_ch1 = np.arange(1, activity_ch1.shape[0] + 1).reshape(-1, 1)
activity_with_timings_ch1 = np.concatenate((timings_ch1, activity_ch1), axis=1)
headers_ch1 = ['timing'] + ['cell' + str(i) for i in range(1, activity_ch1.shape[1] + 1)]
np.savetxt(csv_ch1, activity_with_timings_ch1, delimiter=',', fmt='%.3f', header=','.join(headers_ch1), comments='')
print(f"[✓] mCherry saved to: {csv_ch1}")

# === 2. GCaMP チャネル（channel 2） ===
raw_path_ch2 = {
    "h5_file": r"P:\Tardigrade\Tardigrada_mukumoto\orig\raw_t=517-565\raw_t=517-565.h5",
    "dset": "default",
    "channel": 1
}
activity_ch2 = extract_activities(
    tracked_labels_path=tracked_labels_path,
    raw_path=raw_path_ch2,
    offset_value=100.0,
    do_normalize=False
)
csv_ch2 = os.path.join(path_results, "tardigrade_t=517-565_signals_GCaMP_offset100.csv")
timings_ch2 = np.arange(1, activity_ch2.shape[0] + 1).reshape(-1, 1)
activity_with_timings_ch2 = np.concatenate((timings_ch2, activity_ch2), axis=1)
headers_ch2 = ['timing'] + ['cell' + str(i) for i in range(1, activity_ch2.shape[1] + 1)]
np.savetxt(csv_ch2, activity_with_timings_ch2, delimiter=',', fmt='%.3f', header=','.join(headers_ch2), comments='')
print(f"[✓] GCaMP saved to: {csv_ch2}")

In [1]:
from __future__ import annotations

import os
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
import numpy as np



import re
from glob import glob
from pathlib import Path
from typing import List, Tuple

import numpy as np

from matplotlib.patches import ConnectionPatch

from scipy import ndimage


from CellTracker.stardistwrapper import load_2d_slices_at_time


In [ ]:
from __future__ import annotations

import os
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
import numpy as np



import re
from glob import glob
from pathlib import Path
from typing import List, Tuple

import numpy as np

from matplotlib.patches import ConnectionPatch

from scipy import ndimage


from CellTracker.stardistwrapper import load_2d_slices_at_time


ModuleNotFoundError: No module named 'stardist'

## E. Extract activities from tracked cells

In [2]:
def activities(tracked_labels_path :str, raw_path: str | dict, discard_ratio: float = 0.1, do_normalize=False):
        tracked_labels_path=Path(tracked_labels_path)
        filenames = glob(str(tracked_labels_path / "*t*.tif"))
        assert len(filenames) > 0, f"No labels files were found in {tracked_labels_path / '*t*.tif'}"
        numbers = [int(re.findall(r"t(\d+)", f)[0]) for f in filenames]
        smallest_number = min(numbers)
        largest_number = max(numbers)

        for t in range(smallest_number, largest_number + 1):
            print(f"t={t}...", end="\r")
            try:
                # Load 2D slices at time t
                raw = load_2d_slices_at_time(raw_path, t=t, do_normalize=do_normalize)
            except FileNotFoundError:
                # Handle missing image files
                print(f"Warning: Raw images at t={t - 1} cannot be loaded! Stop calculation!")
                break

            try:
                # Load 2D slices at time t
                labels_img = load_2d_slices_at_time(str(tracked_labels_path / "*t%06i*.tif"), t=t, do_normalize=False)
            except FileNotFoundError:
                # Handle missing image files
                print(f"Warning: Label images at t={t - 1} cannot be loaded!")
                if t == smallest_number:
                    print("Warning: stop calculation!")
                    break
                else:
                    print(f"Warning: skip volume {t - 1}!")
                    activities[t - smallest_number, :] = np.nan
                    continue

            if t == smallest_number:
                cell_num = np.max(labels_img)
                activities = np.zeros((largest_number - smallest_number + 1, cell_num))

            per = (1 - discard_ratio) * 100

            found_bbox = ndimage.find_objects(labels_img, max_label=cell_num)
            for label in range(1, cell_num + 1):
                bbox = found_bbox[label - 1]
                if found_bbox[label - 1] is not None:
                    intensity_label_i = raw[bbox][labels_img[bbox] == label]
                    threshold = np.percentile(intensity_label_i, per)
                    activities[t - smallest_number, label - 1] = np.mean(
                        intensity_label_i[intensity_label_i > threshold])
                else:
                    activities[t - smallest_number, label - 1] = np.nan

        return activities

In [6]:
def activities(tracked_labels_path :str, raw_path: str | dict, do_normalize=False):
    tracked_labels_path = Path(tracked_labels_path)
    filenames = glob(str(tracked_labels_path / "*t*.tif"))
    assert len(filenames) > 0, f"No labels files were found in {tracked_labels_path / '*t*.tif'}"
    numbers = [int(re.findall(r"t(\d+)", f)[0]) for f in filenames]
    smallest_number = min(numbers)
    largest_number = max(numbers)

    for t in range(smallest_number, largest_number + 1):
        print(f"t={t}...", end="\r")
        try:
            # Load raw 2D volume and subtract 100-count offset
            raw = load_2d_slices_at_time(raw_path, t=t, do_normalize=do_normalize)
            raw = raw.astype(np.float32)  # 型変換でunderflow回避
            raw = raw - 100
            #raw = np.maximum(raw, 0)# オフセットによる負値を0に

        except FileNotFoundError:
            print(f"Warning: Raw images at t={t - 1} cannot be loaded! Stop calculation!")
            break

        try:
            labels_img = load_2d_slices_at_time(str(tracked_labels_path / "*t%03i*.tif"), t=t, do_normalize=False)
        except FileNotFoundError:
            print(f"Warning: Label images at t={t - 1} cannot be loaded!")
            if t == smallest_number:
                print("Warning: stop calculation!")
                break
            else:
                print(f"Warning: skip volume {t - 1}!")
                activities[t - smallest_number, :] = np.nan
                continue

        if t == smallest_number:
            cell_num = np.max(labels_img)
            activities = np.zeros((largest_number - smallest_number + 1, cell_num))

        found_bbox = ndimage.find_objects(labels_img, max_label=cell_num)
        for label in range(1, cell_num + 1):
            bbox = found_bbox[label - 1]
            if bbox is not None:
                intensity_label_i = raw[bbox][labels_img[bbox] == label]
                activities[t - smallest_number, label - 1] = np.mean(intensity_label_i)  # ② 全ボクセル平均
            else:
                activities[t - smallest_number, label - 1] = np.nan

    return activities


### Extract activities

### Parameters
- `raw_path`: A string specifying the file path to the raw image to extract the activities. The image data should be in TIFF format.

### Notes
> This Code Block extracts activities from 3D time-lapse images, based on tracking results.

In [8]:
raw_path = {"h5_file":r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=1-225\raw_t=1-225.h5",
            "dset": "default",
            "channel": 1}
# raw_path = r"P:\Ascidiacea\orig\241025ホヤ\20220223-170215ASAPaiviatif_sequence"

tracked_labels_path =r"H:\kumamushi-new\t=0-224"

activities_ch1 = activities(tracked_labels_path = tracked_labels_path, raw_path=raw_path)

TypeError: 'numpy.float32' object cannot be interpreted as an integer

### Save activities into csv

In [4]:
csv_filename = "tardigrade_t=566-665_signals_GCaMP_2.csv"
path_results=r"P:\Tardigrade\Tardigrada_mukumoto\ana\t=566~665"


csv_filename = os.path.join(path_results, csv_filename)
timings = np.arange(1, activities.shape[0]+1).reshape(-1, 1)
activities_with_timings = np.concatenate((timings, activities), axis=1)
headers = ['timing'] + ['cell' + str(i) for i in range(1, activities.shape[1]+1)]
np.savetxt(csv_filename, activities_with_timings, delimiter=',', fmt='%.3f', header=','.join(headers), comments='')
print(f"The cell activities have been saved in {csv_filename}")

#print(activities[2][3])

#np.save("activity_1220",activities )


The cell activities have been saved in P:\Tardigrade\Tardigrada_mukumoto\ana\t=566~665\tardigrade_t=566-665_signals_GCaMP_2.csv


## 細胞間の移動ベクトルの算出

In [ ]:
from scipy.spatial import distance
cell_movement = np.zeros((center_of_labels.shape[0],center_of_labels.shape[1],3),)

for t in range(center_of_labels.shape[0]-1):
    for label in range(center_of_labels.shape[1]):
        if np.isnan(center_of_labels[t+1,label]).any():
            center_of_labels[t+1,label] = center_of_labels[t,label]
        cell_position_t1 = center_of_labels[t,label]
        cell_position_t2 = center_of_labels[t+1,label]
        x = cell_position_t2[0]-cell_position_t1[0]
        y = cell_position_t2[1]-cell_position_t1[1]
        z = cell_position_t2[2]-cell_position_t1[2]
        cell_movement[t, label,0] = x
        cell_movement[t, label,1] = y
        cell_movement[t, label,2] = z


In [ ]:
csv_filename = "cell_movement_vectors.csv"

csv_filename = os.path.join(path_results, csv_filename)

print(f"The cell movements have been saved in {csv_filename}")
np.savetxt(csv_filename, cell_movement.reshape(cell_movement.shape[0], -1), delimiter=',', fmt='%d')

In [1]:
#t=79のみ
import h5py
import numpy as np
from scipy import ndimage


In [4]:
import h5py
import numpy as np
from scipy import ndimage

# === ファイルパス ===
gcamp_path = r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_000.h5"
mcherry_path = r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_001.h5"
label_path = r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_002.h5"

# === HDF5ファイルからデータ読み込み ===
with h5py.File(gcamp_path, 'r') as f:
    gcamp_img = f['data'][()]
with h5py.File(mcherry_path, 'r') as f:
    mcherry_img = f['data'][()]
with h5py.File(label_path, 'r') as f:
    label_img = f['data'][()].astype(np.int32)

# === ラベル数の確認 ===
cell_num = np.max(label_img)
print(f"Detected {cell_num} labeled cells")

# === 活動量抽出関数 ===
def extract_activity(raw_img, labels_img, discard_ratio=0.5):
    activities = np.zeros(cell_num)
    found_bbox = ndimage.find_objects(labels_img, max_label=cell_num)
    per = (1 - discard_ratio) * 100
    for label in range(1, cell_num + 1):
        bbox = found_bbox[label - 1]
        if bbox is not None:
            intensity_label_i = raw_img[bbox][labels_img[bbox] == label]
            threshold = np.percentile(intensity_label_i, per)
            activities[label - 1] = np.mean(intensity_label_i[intensity_label_i > threshold])
        else:
            activities[label - 1] = np.nan
    return activities

# === 抽出実行 ===
gcamp_activities = extract_activity(gcamp_img, label_img, discard_ratio=0.5)
mcherry_activities = extract_activity(mcherry_img, label_img, discard_ratio=0.5)

# === 結果表示 ===
print("Cell\tGCaMP6s\tmCherry")
for i, (g, m) in enumerate(zip(gcamp_activities, mcherry_activities), start=1):
    print(f"{i:02d}\t{g:.2f}\t{m:.2f}")


Detected 2 labeled cells
Cell	GCaMP6s	mCherry
01	142.97	171.59
02	144.30	165.63


In [3]:
import h5py

def list_h5_keys(filepath):
    with h5py.File(filepath, 'r') as f:
        print(f"Contents of {filepath}:")
        f.visititems(lambda name, obj: print(f"  {name} - {type(obj)}"))

list_h5_keys(r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_000.h5")
list_h5_keys(r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_001.h5")
list_h5_keys(r"\\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_002.h5")




Contents of \\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_000.h5:
  data - <class 'h5py._hl.dataset.Dataset'>
Contents of \\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_001.h5:
  data - <class 'h5py._hl.dataset.Dataset'>
Contents of \\4-307-km\p\Tardigrade\Tardigrada_mukumoto\orig\raw_t=0~225\raw-label-t=79_002.h5:
  data - <class 'h5py._hl.dataset.Dataset'>
